In [2]:
%pip install plotly
%pip install plotly_express
%pip install dash
%pip install dash-bootstrap-components
%pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 KB 9.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.4 MB/s eta 0:00:00


In [22]:
from jupyter_dash import JupyterDash
from dash import dcc, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px
import pandas as pd

# Carregando o dataset de espécies de íris
dataset = px.data.iris()
especies = dataset.groupby(['species']).count().reset_index()

# Criando o app onde colocaremos o nosso dashboard
# Vamos inicializar o gráfico de barras, descrito na próxima seção
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])
figBarra = px.bar(especies, 'species', 'species_id', labels={"species":"Espécie","species_id":"Contagem"})

# Construindo componentes do dashboard
# Teremos um título estático, dois dropdowns para escolha de eixos X e Y que afetam o gráfico de dispersão
# Também teremos um gráfico de barras com uma contagem as espécies de íris dos pontos selecionados no gráfico de dispersão
titulo  = dcc.Markdown(children='# \n# Comparando dimensões de íris')
dropdownX = dcc.Dropdown(options=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
                        value='sepal_length', clearable=False)
dropdownY = dcc.Dropdown(options=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
                        value='sepal_width', clearable=False)
graficoScatter = dcc.Graph(figure={})
graficoBarra = dcc.Graph(figure=figBarra)

# Customizando disposição dos componentes
app.layout = dbc.Container([
    dbc.Row([dbc.Col([titulo], width={'size':6,'offset':1})], justify='center', align='center'),
    dbc.Row([dbc.Col([dropdownX], width={'size':4,'offset':2}), dbc.Col([dropdownY], width=4)], align='center'),
    dbc.Row([dbc.Col([graficoScatter], width=6), dbc.Col([graficoBarra], width=6)])
], fluid=True)

# Definindo o callback da escolha de eixos
# Neste callback, o gráfico de dispersão é um output e os dropdowns são inputs
# O usuário decidirá as variáveis dos eixos e o gráfico será atualizado com elas
@app.callback(
    Output(graficoScatter, 'figure'),
    Input(dropdownX, 'value'),
    Input(dropdownY, 'value')
)
def atualizarGraficoScatter(x, y):
    fig1 = px.scatter(dataset, dataset[x], dataset[y], color='species')
    return fig1

# Definindo o callback da seleção de dados
# Neste callback, o gráfico de barras é um output e dados selecionados do gráfico de dispersão são inputs
# O usuário selecionará pontos no gráfico de dispersão e o gráfico de barras mostrará a contagem das espécies de íris
@app.callback(
    Output(graficoBarra, 'figure'),
    Input(graficoScatter, 'selectedData') # Para definirmos dados selecionados em gráfico como input, o plotly usa o argumento 'selectedData'
)
def atualizarGraficoBarra(pontos):  
    if(pontos and len(pontos['points']) > 0):
        # O argumento selectedData não contém um DataFrame, mas uma lista de dicionários com informações de cada ponto do gráfico
        # Essas informações não incluem a classificação dos pontos nominalmente, mas curveNumber pode nos ajudar com isso
        # O curveNumber é um inteiro referente à classificação dos pontos, como por exemplo um diferencial de cor
        # Ele começa em 0 e respeita a ordem das classificações no gráfico de dispersão
        # Sendo setosa a primeira classificação, seu curveNumber correspondente será 0
        curveNumbers = pd.Series([item['curveNumber'] for item in pontos['points']]) # Lembrando: curveNumber é uma chave de dicionário dentro de uma lista de pontos (points)
        curveNumbers = curveNumbers.map({0:'setosa', 1:'versicolor', 2:'virginica'}) # Para o rótulo ser compreensível, vamos traduzir os curveNumbers para as espécies
        pointIndexes = pd.Series([item['pointIndex'] for item in pontos['points']]) # Essa etapa não é necessária, mas queria ter uma outra variável para realizar a contagem
        dataAux = {'species':curveNumbers, 'species_id':pointIndexes} # Tendo duas Series diferentes, basta converter para um dicionário e criar o DataFrame
        dfAux = pd.DataFrame(dataAux)
        dfPlot = dfAux.groupby(['species']).count().reset_index()
        fig2 = px.bar(dfPlot, 'species', 'species_id', labels={"species":"Espécie","species_id":"Contagem"})
    else:
        fig2 = px.bar(especies, 'species', 'species_id', labels={"species":"Espécie","species_id":"Contagem"})
    return fig2

# Rodando o app
if __name__=='__main__':
    app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>